In [79]:
import numpy as np
from natsort import natsorted
import skimage
import time, os, sys
import mxnet as mx
from mxnet import gluon, gpu, nd, kv, cpu
from mxnet.gluon import nn
from mxnet.gluon.model_zoo import vision
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import glob
import sys
sys.path.insert(0,'/github/cellpose/')
from cellpose import models, utils, plot
import imp
imp.reload(utils)
imp.reload(models)

# going to use an example the images from the cellpose website
root_dir = '../web/static/images/' # image directory
exts = ['*.png', '*.tiff', '*.tif', '*.jpg', '*.jpeg'] # extensions of images

# list of files
image_files = []
for e in exts:
    image_files.extend(glob.glob(root_dir + 'img' + e)) # I only want 'img' files
    #image_files.extend(glob.glob(root_dir + e))
image_files = natsorted(image_files)
nimg = len(image_files)
# name of files
files = [os.path.splitext(os.path.split(image_files[i])[1])[0] for i in range(nimg)]
print(image_files)

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
channels = [[2,3], [0,0], [2,3], [0,0], [2,3], [0,0], [2,3], [0,0],
            [0,0], [2,3], [0,0], [2,3], [0,0], [0,0], [0,0], [0,0], 
            [2,3], [0,0], [2,3], [0,0], [2,3], [0,0], [0,0]]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

imgs = [skimage.io.imread(image_files[i]) for i in range(nimg)]

# check if GPU working, and if so use it
use_gpu = utils.use_gpu()
if use_gpu:
    device = mx.gpu()
else:
    device = mx.cpu()

model = models.Cellpose(device)

# if rescale is set to None, the size of the cells is estimated on a per image basis
# if you want to set the size yourself, set it to 27. / average_cell_diameter
masks, flows, styles, diams = model.eval(imgs, rescale=1.0, channels=channels)

for idx in range(nimg):
    img = imgs[idx]
    maski = masks[idx]
    flowi = flows[idx][0]


    fig = plt.figure(figsize=(12,3))
    # can save images (set save_dir=None if not)
    plot.show_segmentation(fig, img, maski, flowi, save_path='../web/static/segs/'+files[idx])
    plt.tight_layout()
    plt.show()

['../web/static/images/img00.png', '../web/static/images/img01.png', '../web/static/images/img02.png', '../web/static/images/img03.png', '../web/static/images/img04.png', '../web/static/images/img05.png', '../web/static/images/img06.png', '../web/static/images/img07.png', '../web/static/images/img08.png', '../web/static/images/img09.png', '../web/static/images/img10.png', '../web/static/images/img11.png', '../web/static/images/img12.png', '../web/static/images/img13.png', '../web/static/images/img14.png', '../web/static/images/img15.png', '../web/static/images/img16.png', '../web/static/images/img17.png', '../web/static/images/img18.png', '../web/static/images/img19.png', '../web/static/images/img20.png', '../web/static/images/img21.png', '../web/static/images/img22.png']


  0%|          | 0/23 [00:00<?, ?it/s]

estimated cell diameters for all images


100%|██████████| 23/23 [00:18<00:00,  1.22it/s]


In [19]:
diams

array([30.52860475, 26.75817632, 26.07678142, 29.81610303, 26.98147513,
       27.47726333, 27.31300057, 26.47640459, 28.35489376, 28.35489376,
       25.94224354, 28.24889378, 28.24889378, 28.01776052, 26.0288284 ,
       27.07397274, 27.09203206, 26.48570904, 27.05549852, 26.29637212,
       26.60826939, 29.61414247, 27.27635723])